In [1]:
import tensorflow.keras as keras
import tensorflow as tf
import matplotlib

# from matplotlib import style
import pandas as pd
import numpy as np

import csv

In [15]:
# keras data example
# 98u7
# mnist = tf.keras.datasets.mnist
# (x_train, y_train),(x_test, y_test) = mnist.load_data()

# x_train = tf.keras.utils.normalize(x_train, axis=1)
# test = tf.keras.utils.normalize(x_test, axis=1)
# plt.imshow(x_train[0],cmap=plt.cm.binary)
# plt.show()

In [2]:
# opens the data and turns it into CSV file
txt_file = "2017-2018season.txt"
csv_file = "2017-2018season.csv"
in_txt = csv.reader(open(txt_file, "r"), delimiter = ',')
season_csv = csv.writer(open(csv_file, 'w'))
season_csv.writerows(in_txt)

season_arrays = []

# turns data into a 2d array called season_arrays which will be the
# raw data that we are trying to process. 
with open('2017-2018season.csv', newline='') as season_csv:
    reader = csv.reader(season_csv, delimiter=' ', quotechar='|')
    for rows in reader:
        for item in rows:
            row = item.split(',')
            #numerating away/home
            if row[3] == 'Away':
                row[3] = 0
            elif row[3] == 'Home':
                row[3] = 1
        if row in season_arrays:
            continue
        else:
            season_arrays.append(row)
            
# had an idea to split the home and away into separate arrays and combining them later, but 
# decided not to use them. they are just here in case I find a use for them but you can ignore for now
# unless of course you can think of a simple way to use them
home = []
away = []
for row in season_arrays:
    if row[3] == 1:
        home.append(row)
    else: away.append(row)
        
# defining categories and a dict for turning all of the team names into ints (just more processing strings to nums)
home_team_to_predict = "DET"
away_team_to_predict = "MIN"

categories = ['game_date', 'home_team', 'enemy_team', 'home_away', 'win', 'loss', 'over_time', 'points', 'goals_for',
              'goals_against', 'shots_for', 'shots_against', 'power_play_goals_for', 'power_play_opportunities', 
              'power_play_percent','power_play_goals_against', 'penalty_kill_percent', 'faceoff_wins', 'faceoff_losses', 
              'faceoff_win_pct']

teams_dict = {'AFM': 1,'ANA': 2,'ARI': 3,'ATL': 4,'BOS': 5,'BRK': 6,'BUF': 7,'CAR': 8,'CGS': 9,'CGY': 10,'CHI': 11,
              'CBJ': 12,'CLE': 13,'CLR': 14,'COL': 15,'DAL': 16,'DFL': 17,'DCG': 18,'DET': 19,'EDM': 20,'FLA': 21,
              'HAM': 22,'HFD': 23,'KCS': 24,'LAK': 25,'MIN': 26,'MMR': 27,'MNS': 28,'MTL': 29,'MWN': 30,'NSH': 31,
              'NJD': 32,'NYA': 33,'NYI': 34,'NYR': 35,'OAK': 36,'OTT': 37,'PHI': 38,'PHX': 39,'PIR': 40,'PIT': 41,
              'QUA': 42,'QUE': 43,'QBD': 44,'SJS': 45,'SLE': 46,'STL': 47,'TAN': 48,'TBL': 49,'TOR': 50,'TSP': 51,
              'VAN': 52,'VGK': 53,'WPG': 54,'WSH': 55}

In [3]:
# creation of the pandas dataframe 
season_df = pd.DataFrame(season_arrays, columns=categories)
game_dates = season_df['game_date']
unique_dates = season_df['game_date'].unique()

# turns dates into values (i.e. 1,2,3,4) so that the sequentiality is clear to the neural net
# I could be wrong in doing this, so just be wary of that. 
def dates_to_val(array, dates_array):
    array = array.tolist()
    dates_array = dates_array.tolist()
    new_dates = []
    for i in enumerate(array):
        for j in enumerate(dates_array):
            if i[1] == j[1]:
                new_dates.append(dates_array.index(j[1]))
    return new_dates

# maping data to the DF
new_dates = dates_to_val(game_dates,unique_dates)
season_df['home_team'] = season_df['home_team'].map(teams_dict)
season_df['enemy_team'] = season_df['enemy_team'].map(teams_dict)
season_df['game_date'] = new_dates

# categories that I would like to have differentiated (not all of them ended up being used but this is still here as reference point)
differential_categories = ['PPvsPK', 'PPGvsPPGA', 'faceoff_WINSvsLOSS', 'faceoff_LOSSvsLWINS', 'penalty_kill_diff']

# turning some of the info into variables for ease of use 
pp = season_df['power_play_percent']
pk = season_df['penalty_kill_percent']
ppg = season_df['power_play_goals_for']
ppga = season_df['power_play_goals_against']
foW = season_df['faceoff_wins']
foL = season_df['faceoff_losses']

# turns text into int or floats respectively
def num(s):
    try:
        return int(s)
    except ValueError:
        return float(s)

# finds the difference between two pieces of data. 
# only a couple of statistics can be differentiated using this method and that is all we will be doing for now.
def differentiate(list1, list2):
    new = []
    if len(list1) == len(list2):
        if (isinstance(num(list1[0]), float)):
            for i,j in enumerate(list1):
                new.append(round(float(list2[i]) - float(j), 3))
        else:
            for i,j in enumerate(list1):
                new.append(num(list2[i]) - num(j))
    else:
        print(f"------List lengths do not match-----\n List 1 length: {len(list1)} \n list 2 length: {len(list2)}")
    return np.array(new)

# one dimensional differentials 
# differentials_list = differentiate(pp,pk), differentiate(ppg,ppga), differentiate(foW,foL)

# two dimensional differentials - will add later but I think we have enough data to begin actual pre-processing for NN
# season_df['faceoff_LOSSvsLWINS']
# season_df['penalty_kill_diff']

# creates differential statistics and adds them to the season_df 
season_df['PPvsPK'] = differentiate(pp,pk)
season_df['PPGvsPPGA'] =  differentiate(ppg,ppga)
season_df['faceoff_WvsL'] = differentiate(foW,foL)

# what we have here is the full coallition of numerical data all put together into one databse. 
# this is what we are working from
print(season_df.head())

   game_date  home_team  enemy_team  home_away win loss over_time points  \
0          0         38          45          0   1    0         0      2   
1          0         50          54          0   1    0         0      2   
2          0         20          10          1   1    0         0      2   
3          0         47          41          0   1    0         0      2   
4          0         41          47          1   0    0         1      1   

  goals_for goals_against     ...      power_play_opportunities  \
0         5             3     ...                             5   
1         7             2     ...                             4   
2         3             0     ...                             2   
3         5             4     ...                             4   
4         4             5     ...                             5   

  power_play_percent power_play_goals_against penalty_kill_percent  \
0               60.0                        2                 66.7   


In [4]:
# this is where things get messy. this is where I have attempted to combine arrays for one game into one single array

# finds the data for one whole game and returns the info in one array (not yet formatted properly)
def get_pair(df,date,home,away):
    print(type(df),type(date),type(home),type(away))
    home_p = season_df.loc[season_df["home_team"] == home].loc[season_df["game_date"] == date]
    enemy_p = season_df.loc[season_df["home_team"] == away].loc[season_df["game_date"] == date]
    return [home_p, enemy_p]

# season_df.loc[season_df["home_team"] == 47].loc[season_df["game_date"] == 0]
# type(season_df.loc[season_df["home_team"] == 41].loc[season_df["game_date"] == 0])

type(get_pair(season_df,0,47,41))

# def final_sort(df):
    
    

# loop through
#  if names and name reciprocal and date match
#   calc differentials 
#    pair them and push to array for that date
#     save arrays as df 





# print(season_df)   
# differentials_df = pd.DataFrame(differentials, columns=differentials)


<class 'pandas.core.frame.DataFrame'> <class 'int'> <class 'int'> <class 'int'>


list

In [10]:
from pandas import read_csv
from matplotlib import pyplot
# # load dataset
# dataset = season_df
# values = dataset.values
# # specify columns to plot
# groups = [1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
# i = 1
# # plot each column
# pyplot.figure()
# for group in groups:
# 	pyplot.subplot(len(groups), 1, i)
# 	pyplot.plot(values[:, group])
# 	pyplot.title(dataset.columns[group], y=2, loc='right')
# 	i += 1
# # pyplot.autoscale(enable=True, axis='both', tight=None)
# pyplot.savefig('nigggggs.png', bbox_inches='tight')
# pyplot.show()

# season_df.groupby('home_away').plot(kind='bar', x='game_date')
# pyplot.show()
# season_df.plot(kind='bar', x='game_date')

In [ ]:
##### create data splitter that can pul sequenital past x games on two different teams 



## combine data into columns by game 
# detect two lines of same game, determine if game has been done already, if not combine and write 
# for row in df.iterrows():
#     buffer_df[0]
# print(season_arrays)

In [ ]:
## balance, scale, and create training data (~80%)
## normalize the batches
# def normalize_with_moments(x, axes=[0, 1], epsilon=1e-8):
#     mean, variance = tf.nn.moments(x, axes=axes)
#     x_normed = (x - mean) / tf.sqrt(variance + epsilon) # epsilon to avoid dividing by zero
#     return x_normed

# def parse_example(line_batch):
#     # ...
#     features = tf.transpose(features)
#     features = normalize_with_moments(features)
#     # ...
# also create out of sampple data (~20%)

In [10]:
# input nodes that take 2x sets of data (one game per node) by format of out of sample data

In [17]:
#  train model 
# careful of the input data overlapping training data 